In [1]:
# Instalando as bibliotecas para o web scraping

%pip install selenium
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importando as Bibliotecas

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing.dummy import Pool as ThreadPool

In [3]:
def pegar_conteudo(url):
    dados_curso = [url.split('/')[4]]

    driver = webdriver.Firefox(options=opcoes)
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CourseAbout__header__title')) and EC.presence_of_element_located((By.CLASS_NAME, 'CourseAbout__header__info')))

        conteudo = BeautifulSoup(driver.page_source, 'html.parser')    
        driver.quit()
        try:
            nome_curso = conteudo.find('h1', class_='CourseAbout__header__title').text
        except:
            nome_curso = ''

        if len(nome_curso) == 0:
            nome_curso = 'Nome não disponível'
        dados_curso.append(nome_curso)

        try:
            num_alunos = int(conteudo.find('div', class_='CourseAbout__header__info CourseAbout__header__info--students').text[:-7])
        except:
            num_alunos = 'Número de alunos não disponível'
        dados_curso.append(num_alunos)

        return dados_curso
    except:
        driver.quit()
        return None

def pegar_dados(curso_max):

    base_url = 'https://app.sapium.com.br/course/{}/about'
    urls = [base_url.format(i) for i in range(1000, curso_max)]

    global opcoes
    opcoes = webdriver.FirefoxOptions()

    pool = ThreadPool(8)
    dados_cursos = pool.map(pegar_conteudo, urls)
    pool.close()
    pool.join()

    dados_cursos = [dado for dado in dados_cursos if dado]

    cursos_df = pd.DataFrame(dados_cursos, columns=['Número', 'Nome', 'Número de Alunos'])
    cursos_df = cursos_df.set_index('Número')

    return cursos_df

cursos_df = pegar_dados(3000)
cursos_df

,Nome,Número de Alunos
Número,,
1008,Nome não disponível,72
1013,SAUDAÇÕES EM ESPANHOL - AULA GRATUITA,1751
1016,Curso de Pilates - MAT PILATES 2.0,477
1026,Curso de Day Trade na Prática: Domine o Mercad...,Número de alunos não disponível
1038,Curso de Osteologia e Anatomia Radiológica do ...,Número de alunos não disponível
...,...,...
2938,Curso de Matemática Básica: Dicas Poderosas e ...,Número de alunos não disponível
2943,PTAM - Parecer Técnico de Avaliação Mercadológica,Número de alunos não disponível
2951,GEOPROCESSAMENTO COM O USO DO QGIS,4990


In [8]:
for i in range(1, len(cursos_df["Número de Alunos"])):
    try:
        if cursos_df["Número de Alunos"][i] > 1000:
            print(cursos_df.iloc[i])
    except:
        pass

Nome                SAUDAÇÕES EM ESPANHOL - AULA GRATUITA
Número de Alunos                                     1751
Name: 1013, dtype: object
Nome                Curso Produção de Imagens e Vídeos para Profes...
Número de Alunos                                                 3197
Name: 1129, dtype: object
Nome                INTENSIVO DE ESPANHOL ARGENTINO A1-B2  COM PRE...
Número de Alunos                                                 1127
Name: 1330, dtype: object
Nome                KIT - Método de Vendas Exponencial para Cursos
Número de Alunos                                              1346
Name: 1626, dtype: object
Nome                Geração de Mapas Digitais com o Google
Número de Alunos                                      1851
Name: 1630, dtype: object
Nome                COMUNICAÇÃO CARTOGRÁFICA E A LINGUAGEM DOS MAPAS
Número de Alunos                                                1703
Name: 1905, dtype: object
Nome                GEOPROCESSAMENTO COM O USO DO QGIS
Núm